# All mammals grouped by institution

In [1]:
import sys
sys.path.append('..')

In [2]:
import csv
import re
from pathlib import Path
from pprint import pp
import json
from collections import defaultdict
from ipywidgets import interact
from pprint import pprint
from tqdm import tqdm
import pandas as pd

In [3]:
DATA_DIR = Path('..') / 'data'
OUT_DIR = DATA_DIR / 'output'

BASE_NAME = 'all_mammals_2021-11-09'
VER = 'd'

JSONL = OUT_DIR / f'{BASE_NAME}.jsonl'
DETAIL = OUT_DIR / f'{BASE_NAME}{VER}.csv'
COUNTS = OUT_DIR / f'{BASE_NAME}{VER}_counts.csv'

VERTNET = DATA_DIR / "vertnet_latest_mammals.csv"

## Read data

### Read previous data

In [4]:
df = pd.read_json(JSONL, lines=True)
df.shape

(649163, 78)

In [5]:
df.columns

Index(['catalognumber', 'continent', 'country', 'county', 'decimallatitude',
       'decimallongitude', 'eventdate', 'fieldnotes', 'geodeticdatum',
       'georeferenceprotocol', 'georeferenceverificationstatus', 'locality',
       'occurrenceid', 'recordedby', 'reproductivecondition', 'sex',
       'verbatimcoordinates', 'verbatimcoordinatesystem', 'verbatimelevation',
       'verbatimeventdate', 'institutioncode', 'collectioncode',
       'dynamicproperties', 'scientificname', 'body_mass', 'ear_length',
       'hind_foot_length', 'tail_length', 'total_length', 'embryo_count',
       'georeferencesources', 'testes_size', 'habitat', 'lactation_state',
       'georeferencedby', 'georeferenceddate', 'highergeography', 'lifestage',
       'verbatimlocality', 'coordinateuncertaintyinmeters', 'pregnancy_state',
       'locationremarks', 'placental_scar_count', 'placental_scar_state',
       'embryo_length', 'vagina_state', 'countrycode', 'islandgroup',
       'verbatimlatitude', 'verbatimlo

### Read orders, and families

In [6]:
ORDERS = {}
FAMILIES = {}

with open(VERTNET) as f:
    reader = csv.DictReader(f)

    for row in tqdm(reader):
        sci_name = row["scientificname"]

        if order := row["order"]:
            ORDERS[sci_name] = order

        if family := row["family"]:
            FAMILIES[sci_name] = family

4666375it [01:32, 50632.30it/s]


## Get counts

In [7]:
key = "body_measurements"
df[key] = 0
df.loc[df["body_mass"].notna(), key] += 1
df.loc[df["ear_length"].notna(), key] += 1
df.loc[df["hind_foot_length"].notna(), key] += 1
df.loc[df["tail_length"].notna(), key] += 1
df.loc[df["total_length"].notna(), key] += 1
df.loc[df["forearm_length"].notna(), key] += 1
df.loc[df["tragus_length"].notna(), key] += 1

In [8]:
key = "female_counts"
df[key] = 0
df.loc[df["embryo_count"].notna(), key] += 1
df.loc[df["embryo_length"].notna(), key] += 1
df.loc[df["lactation_state"].notna(), key] += 1
df.loc[df["nipple_state"].notna(), key] += 1
df.loc[df["nipples_enlarged"].notna(), key] += 1
df.loc[df["nipple_count"].notna(), key] += 1
df.loc[df["ovaries_state"].notna(), key] += 1
df.loc[df["ovaries_size"].notna(), key] += 1
df.loc[df["placental_scar_count"].notna(), key] += 1
df.loc[df["placental_scar_state"].notna(), key] += 1
df.loc[df["pregnancy_state"].notna(), key] += 1
df.loc[df["vagina_state"].notna(), key] += 1

In [9]:
key = "male_counts"
df[key] = 0
df.loc[df["scrotal_state"].notna(), key] += 1
df.loc[df["testes_size"].notna(), key] += 1
df.loc[df["testes_state"].notna(), key] += 1

## Build data frames

In [10]:
df["order"] = df["scientificname"].map(ORDERS).fillna("")
df["family"] = df["scientificname"].map(FAMILIES).fillna("")

In [11]:
cols = """
    institutioncode catalognumber
    order family scientificname
    body_measurements female_counts male_counts
    """.split()
df = df.loc[:, cols]

In [12]:
df = df.sort_values(["institutioncode", "order", "family", "scientificname"])

In [13]:
df2 = df.drop(columns="catalognumber")
df2 = df2.groupby(["institutioncode", "order", "family", "scientificname"]).agg("sum")
df2.head(20)

body_measurements  \
institutioncode order        family      scientificname                                       
AMNH            Artiodactyla Antilopinae Ourebia ourebi haggardi                          0   
                             Bovidae     Bison bison                                      0   
                                         Bovidae                                          0   
                                         Hemitragus                                       0   
                                         Ovibos moschatus                                 0   
                                         Ovis                                             0   
                                         Ovis ammon poli                                  1   
                                         Syncerus                                         0   
                                         Syncerus caffer nanus                            1   
                                         Taurotragus derbianus gigas                      0   
                                         Tragelaphus                                      0   
                                         Tragelaphus scriptus                             0   
                             Capreolinae Mazama americana sarae                           0   
                             Caprinae    Naemorhedus goral griseus                        1   
                             Cervidae    Elaphurus davidianus                             0   
                                         Mazama                                           0   
                                         Mazama gouazoupira                               0   
                                         Muntiacus muntjak pleiharicus                    3   
                                         Odocoileus hemionus columbianus                  6   
                                         Odocoileus hemionus crooki                       1   

                                                                          female_counts  \
institutioncode order        family      scientificname                                   
AMNH            Artiodactyla Antilopinae Ourebia ourebi haggardi                      1   
                             Bovidae     Bison bison                                  1   
                                         Bovidae                                      1   
                                         Hemitragus                                   0   
                                         Ovibos moschatus                             0   
                                         Ovis                                         2   
                                         Ovis ammon poli                              0   
                                         Syncerus                                     1   
                                         Syncerus caffer nanus                        0   
                                         Taurotragus derbianus gigas                  1   
                                         Tragelaphus                                  1   
                                         Tragelaphus scriptus                         1   
                             Capreolinae Mazama americana sarae                       0   
                             Caprinae    Naemorhedus goral griseus                    0   
                             Cervidae    Elaphurus davidianus                         0   
                                         Mazama                                       0   
                                         Mazama gouazoupira                           1   
                                         Muntiacus muntjak pleiharicus                0   
                                         Odocoileus hemionus columbianus              0   
                                         Odocoileus hemionus crooki     

## Write output

In [14]:
df.to_csv(DETAIL, index=False)

In [15]:
df.shape

(649163, 8)

In [16]:
df2.to_csv(COUNTS)

In [17]:
df2.shape

(21995, 3)